In [2]:
import pandas as pd
import numpy as np
import multiprocessing
import glob

In [3]:
# 3.1. Tratamento para Indicadores com Impacto Negativo no Empreendedorismo

def negative(series):
    
    series = 1/series
    
    return series.replace(np.inf, 0)
    

In [4]:
# 3.2. Tratamento para Observações Faltantes (missing data)

def missing_data(df):
    
    ind = pd.read_excel('Arquivos ICE - 23/Ind_Originais_ICE_2022.xlsx')
    ind.columns = ind.iloc[1]
    ind.columns.values[0] = 'Município'
    ind = ind.set_index('Município')
    ind = ind.tail(101)
    
    for c in df.columns:
        if c in ind.columns:
            if df[c].isna().sum()/len(df) > 0.3:
                df[c].fillna(ind[c], inplace=True)
        
        df[c].fillna(0)
                
    return df


In [5]:
# 3.3. Tratamento para Valores Extremos

def extreme_values(df):
    
    for c in df.columns:
    
        top_values = [i for i in df[c] if i > df[c].quantile(0.98)]
        bottom_values = [i for i in df[c] if i < df[c].quantile(0.02)]
        removed = []

        if len(top_values) > 1:
            for i in top_values:
                nlist = [t for t in top_values if t != i]
                if i >= 5*sum(nlist)/(len(nlist)):
                    removed.append(i)

        if len(top_values) > 1:
            for j in bottom_values:
                if i <= 5*sum(top_values.remove(i))/(len(top_values)-1):
                    removed.append(j)

        for r in df.index:
            if df.loc[r,c] in removed:
                df.iat[r,c] = 0
            
    return df

In [6]:
# 3.4. Padronização de Indicadores

def normalize(series):
    return (series - series.mean())/series.std()

def create_subindex(df, subdet):
    
    i_name = 'Índice de '+subdet
    
    if i_name not in df.columns:
        norm_data = df.apply(lambda x: normalize(x), axis=0)
        df[i_name] = normalize(norm_data.sum(axis=1)) + 6
    
    return df

def create_detindex(df, det):
    
    d_name = 'Índice de ' + det
    det_df = pd.DataFrame()
    
    if d_name not in df.columns:
        for i in (df.columns.levels[0]):
            det_df[i] = df[i,(df[i].columns[-1])]

        det_df = det_df.apply(lambda x: normalize(x), axis=0)
        det_df[d_name] = normalize(det_df.sum(axis=1)) + 6
        df[d_name] = det_df[d_name]
    
    return df

In [7]:
database = pd.DataFrame()
df = {}

In [8]:
# 1. AMOSTRA

amostra = pd.read_csv('AMOSTRA/100-municipios.csv')
database['Município'] = amostra['NOME DO MUNICÍPIO']
database['UF'] = amostra['UF']
database = database.set_index(['Município', 'UF'])

In [ ]:
# 2.2. DETERMINANTE AMBIENTE REGULATÓRIO

# ---------------------------------------------------------------------------------------------
# 2.2.1. SUBDETERMINANTE TEMPO DE PROCESSOS
subdet = 'Tempo de processos'

def read_excel(filenames):
    return pd.read_excel(filenames, header = 1, usecols = "I,R,AA")
    
if __name__ == "__main__":
    files = glob.glob("DETERMINANTE AMBIENTE REGULATÓRIO/REDESIM/*.xlsx")

    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        data = pool.map(read_excel, files)
        sub_tempo_pro = pd.concat(data)

# ---------------------------------------------------------------------------------------------
# 2.2.2. SUBDETERMINANTE TRIBUTAÇÃO


In [ ]:
# 2.8. DETERMINANTE CULTURA

def save_googletrends(database, term):
    
    name = term.split(' ')[-1].replace('_', ' ')
    term = term.replace('_', ' ')
    c_name = 'Pesquisas '+term
    
    if c_name not in database.columns:
        indicador = pd.read_csv('DETERMINANTE CULTURA/geoMap-'+name+'.csv').reset_index()
        indicador = indicador.rename(columns={'index':'Município', 'Category: All categories': c_name})
        database = database.merge(indicador, how='left',on='Município').fillna(0)
        database[c_name] = database[c_name].astype(int)
    
    return database

cultura = {}

# ---------------------------------------------------------------------------------------------
# 2.8.1. SUBDETERMINANTE INICIATIVA

subdet = 'Iniciativa'

iniciativa = ['pelo Termo Empreendedora', 'pelo Termo Empreendedorismo', 'pelo Termo MEI']
sub_iniciativa = pd.DataFrame()
sub_iniciativa.index = database.index

for i in iniciativa:
    sub_iniciativa = save_googletrends(sub_iniciativa, i).set_index('Município')
    
missing_data(sub_iniciativa)
extreme_values(sub_iniciativa)
create_subindex(sub_iniciativa, subdet)
cultura[subdet] = sub_iniciativa

# ---------------------------------------------------------------------------------------------
# 2.8.2. SUBDETERMINANTE INSTITUIÇÕES

subdet = 'Instituições'

instituicoes = ['por Sebrae', 'por Franquia', 'por SIMPLES_Nacional', 'por Senac']
sub_instituicoes = pd.DataFrame()
sub_instituicoes.index = database.index

for i in instituicoes:
    sub_instituicoes = save_googletrends(sub_instituicoes, i).set_index('Município')

missing_data(sub_instituicoes)
extreme_values(sub_instituicoes)
create_subindex(sub_instituicoes, subdet)
cultura[subdet] = sub_instituicoes

# ---------------------------------------------------------------------------------------------
cultura = pd.concat(cultura, axis=1)

df['Cultura'] = cultura
df = pd.concat(df, axis=1)
df

In [ ]:
df